## Import Libraries

In [7]:
import pandas as pd
import plotly.express as px
import numpy as np
from darts import TimeSeries
from darts.models import TiDEModel
from darts.dataprocessing.transformers import Scaler
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.utils.likelihood_models import QuantileRegression
from darts.dataprocessing.transformers import StaticCovariatesTransformer

## Global Variables

In [8]:
TIME_COL = "Date"
TARGET = "Weekly_Sales"
STATIC_COV = ["Store", "Dept"]
FREQ = "W-FRI"
SCALER = Scaler()
TRANSFORMER = StaticCovariatesTransformer()

## Read and Transform data into Darts Format

train = pd.read_csv('data/train.csv')
train['id'] = train['Store'].astype(str)+'-'+train['Dept'].astype(str)

series = train.groupby('id').agg({'Date':'max'}).reset_index()
series = series[series['Date'] == max(train['Date'])]['id'].tolist()

train = train[train['id'].isin(series)]

train.to_csv('data/train_filtered.csv', index=False)

In [9]:
# read train and test datasets and transform train dataset
train = pd.read_csv('data/train_filtered.csv')

train["Date"] = pd.to_datetime(train["Date"])
train[TARGET] = np.where(train[TARGET] < 0, 0, train[TARGET])
train_darts = TimeSeries.from_group_dataframe(df=train, group_cols=STATIC_COV, time_col=TIME_COL, value_cols=TARGET, freq=FREQ, fill_missing_dates=True, fillna_value=0)

# read test dataset and determin Forecast Horizon
test = pd.read_csv('data/test.csv')
test["Date"] = pd.to_datetime(test["Date"])
FORECAST_HORIZON = len(test['Date'].unique())

## Create Dynamic Covariates

In [10]:
# we get the holiday data that we have in both train and test dataset
holidays_df = pd.concat([train[["Date", "IsHoliday"]], test[["Date", "IsHoliday"]]]).drop_duplicates()
# convert bool to numeric
holidays_df["IsHoliday"] = holidays_df["IsHoliday"]*1


In [11]:
# create dynamic covariates for each serie in the training darts
dynamic_covariates = []
for serie in train_darts:
    # add the month and week as a covariate
    covariate = datetime_attribute_timeseries(
        serie,
        attribute="month",
        one_hot=True,
        cyclic=False,
        add_length=FORECAST_HORIZON,
    )
    covariate = covariate.stack(
        datetime_attribute_timeseries(
            serie,
            attribute="week",
            one_hot=True,
            cyclic=False,
            add_length=FORECAST_HORIZON,
        )
    )
    # create holidays with dates for training and test
    holidays_serie = pd.merge(pd.DataFrame(covariate.time_index).rename(columns={'time':'Date'}), holidays_df, on='Date', how='left')
    covariate = covariate.stack(
                TimeSeries.from_dataframe(holidays_serie, time_col="Date", value_cols="IsHoliday", freq=FREQ)
            )
    dynamic_covariates.append(covariate)

## Transform Data for TiDE

In [12]:
# scale covariates
dynamic_covariates_transformed = SCALER.fit_transform(dynamic_covariates)

# scale data
data_transformed = SCALER.fit_transform(train_darts)

# transform static covariates
data_transformed = TRANSFORMER.fit_transform(data_transformed)

## TiDE

In [16]:
dynamic_covariates_transformed[0]

<TimeSeries (DataArray) (time: 182, component: 65, sample: 1)> Size: 95kB
array([[[0.],
        [1.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [1.],
        [0.],
        ...,
        [0.],
        [0.],
        [1.]],

       [[0.],
        [1.],
        [0.],
        ...,
...
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]]])
Coordinates:
  * time       (time) datetime64[ns] 1kB 2010-02-05 2010-02-12 ... 2013-07-26
  * component  (component) object 520B 'month_1' 'month_2' ... 'IsHoliday'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

In [13]:
TiDE_params = {
    "input_chunk_length": 4, # number of weeks to lookback
    "output_chunk_length": FORECAST_HORIZON,
    "num_encoder_layers": 1,
    "num_decoder_layers": 1,
    "decoder_output_dim": 1,
    "hidden_size": 15,
    "temporal_width_past": 4,
    "temporal_width_future": 4,
    "temporal_decoder_hidden": 26,
    "dropout": 0.1,
    "batch_size": 16,
    "n_epochs": 5,
    "likelihood": QuantileRegression(quantiles=[0.25, 0.5, 0.75]),
    "random_state": 42,
    "use_static_covariates": True,
    "optimizer_kwargs": {"lr": 1e-3},
    "use_reversible_instance_norm": False,
}

model = TiDEModel(**TiDE_params)
model.fit(data_transformed, future_covariates=dynamic_covariates_transformed, verbose=False)
pred = SCALER.inverse_transform(model.predict(n=FORECAST_HORIZON, series=data_transformed, num_samples=50, future_covariates=dynamic_covariates_transformed))

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


TypeError: Cannot convert a MPS Tensor to float64 dtype as the MPS framework doesn't support float64. Please use float32 instead.

In [ ]:
# let's check one example
pred_df = pred[0].quantile_df(0.5).reset_index().rename(columns={'Weekly_Sales_0.5':'forecast'})
pred_df['Store'] = TRANSFORMER.inverse_transform(pred[0]).static_covariates['Store'][0]
pred_df['Dept'] = TRANSFORMER.inverse_transform(pred[0]).static_covariates['Store'][0]
pred_df = pd.concat([train[(train['Store'] == 1) & (train['Dept'] == 1)], pred_df])

In [ ]:
fig = px.line(
    pred_df,
    x=pred_df["Date"],
    y=['Weekly_Sales', 'forecast'],
    hover_data={"Date": "|%B %d, %Y"},
    width=1350,
    height=500,
)

fig.update_layout(
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01),
    yaxis_title="Weekly Sales",
    xaxis_title="Delivery Week",
)

fig.show()